In [1]:
import pandas as pd

In [2]:
client_info_df = pd.read_csv(r'E:\wd_work\unti_cheat\D1_B_0201_0613_0615_B.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#######城市变更次数########################

In [6]:
client_info_df.ACTION_CONTROL.value_counts()

my_home_address        815662
my_company_name        524858
my_contact_phone       376697
my_contact             319647
my_work_address        137682
city                    30662
my_phone                12943
my_contact_relation      9442
Name: ACTION_CONTROL, dtype: int64

In [8]:
client_city_df = client_info_df[client_info_df.ACTION_CONTROL=='city']

In [11]:
client_city_df.columns

Index(['ACTION_CONTROL', 'ACTION_GROUP', 'ACTION_PAGE', 'ACTION_TEXT',
       'ACTION_VALUE', 'CLIENT_TIME', 'CREATE_DATE', '中文匹配', 'D3', 'D10', 'F6',
       'GETTEL(A.MOBILE)', 'ID', 'MACHINE_UUID', 'MAX_MAXOVERDUEDAYS', '记录数',
       'PUTOUTDATE_TIMES', 'T3%', 'T30#', 'T3#', 'T10#', '[T3->T10]%', 'T10% ',
       'T20#', 'TEL', 'USER_ID', '日(APPLY_DATE)', '日(MIN_PUTOUTDATE)'],
      dtype='object')

In [15]:
city_changed_cnt = client_city_df.drop_duplicates(['USER_ID','ACTION_VALUE'])[['USER_ID','ACTION_VALUE']].groupby('USER_ID').count().reset_index()

In [ ]:
#########详细地址变更次数########################

In [212]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [ ]:
# difflib.SequenceMatcher(None,str(addr1),str(addr_list[j])).ratio()

In [213]:

import re
# newpat=''
# s = '北一中路19号沈阳国际shang？wu'
# news=re.sub(u"[a-z？]",newpat,s)
# def sub_str(text):
#     newpat = ''
#     return re.sub(u"[a-z？]",newpat,text)
client_addr_df.ACTION_VALUE = client_addr_df.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z？]",'',str(x)))

D:\Anaconda\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [215]:
client_addr = []
user_idlist = []
client_addr_df['str_len'] = client_addr_df.ACTION_VALUE.str.len()
client_addr_df['str_len1'] =client_addr_df['str_len'].shift(-1).fillna(list(client_addr_df['str_len'])[-1])
client_addr_df['str_len_dif'] = client_addr_df['str_len1']-client_addr_df['str_len']
client_addr_df['str_len_dif1'] =client_addr_df['str_len_dif'].shift(1)
for user_id in set(list(client_addr_df.USER_ID)):
#     print(user_id)
    aa = client_addr_df[client_addr_df.USER_ID==user_id]
    aaaa = aa.iloc[1:-1,:]
    aaa = aaaa[(aaaa.str_len_dif1>=0) & (aaaa.str_len_dif<0)]
    if len(aaa)>=1:
        bb = list(aaa.ACTION_VALUE)
        bb.append(list(aa.iloc[-1:,:].ACTION_VALUE)[-1])
        client_addr.append(bb)
    else:
        client_addr.append(list(aa.iloc[-1:,:].ACTION_VALUE)[-1])
    user_idlist.append(user_id)

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [216]:
client_addr1 = []
for addr in client_addr:
    aaaaa = []
    if type(addr) != str:       
        for addr1 in addr:
#             print(len(addr1))
            if len(addr1)>5:
                aaaaa.append(addr1)
        client_addr1.append(aaaaa)
    else:
        if len(addr)>5:
            client_addr1.append(addr)
        else:
            client_addr1.append(' ')

In [217]:
id_list = []
c = 0
for addr in client_addr1:
    
    if type(addr) != str and len(addr)>1:       
        id_list.extend([user_idlist[c],]*len(addr))
        
    elif type(addr) == str or len(addr)==1:
        id_list.append(user_idlist[c])
    c += 1
addr_list = []
for addr in client_addr1:
    if type(addr) != str:       
        addr_list.extend(addr)
        
    elif type(addr) == str :
        addr_list.append(addr)
           

In [218]:
ADDR_DF = pd.DataFrame({'USER_ID':id_list,'ADDR':addr_list})

In [219]:
ADDR_DF['ADDR_LEN'] = ADDR_DF.ADDR.str.len()

In [220]:
ADDR_DF = ADDR_DF.sort_values(by=['USER_ID','ADDR_LEN'],ascending=False)

In [221]:
addr_cnt_df = ADDR_DF.groupby('USER_ID').count().reset_index()[['USER_ID','ADDR']].rename(columns={'ADDR':'addr_cnt'})
ADDR_DF1 = pd.merge(ADDR_DF,addr_cnt_df,on='USER_ID',how='left')

In [222]:
ADDR_DF2 = ADDR_DF1[ADDR_DF1.addr_cnt>1]

In [223]:
ADDR_DF2

,ADDR,USER_ID,ADDR_LEN,addr_cnt
1,吕岭路1004舍,2793140,8,2
2,吕岭路白糖切,2793140,6,2
8,南莲路冶建531号1栋102,2793006,14,4
9,南莲路冶建531号1栋Q,2793006,12,4
10,南莲路冶建503号,2793006,9,4
11,南莲路冶建503,2793006,8,4
26,荣乐中路28弄44号501,2792537,13,6
27,荣乐中路28弄44号401,2792537,13,6
28,荣乐中路28弄45,2792537,9,6
29,荣乐中路38,2792537,6,6


In [224]:
import difflib
data_box = []
for user_id in set(list(ADDR_DF2.USER_ID)):
    ADDR_DF3 = ADDR_DF2[ADDR_DF2.USER_ID==user_id]
    addr1 = list(ADDR_DF3.ADDR)[0]
    for i in range(1,len(ADDR_DF3)):
        if difflib.SequenceMatcher(None,str(addr1),str( list(ADDR_DF3.ADDR)[i])).ratio()<0.4:
            data_box.append((user_id,addr1,list(ADDR_DF3.ADDR)[i]))

In [225]:
ADDR_DF3 = pd.DataFrame(data_box,columns=['USER_ID','ADDR1','ADDR2'])

In [226]:
ADDR_DF3['ADDR_LEN'] = ADDR_DF3.ADDR2.str.len()
ADDR_DF3 = ADDR_DF3.sort_values(by='ADDR_LEN')

In [227]:
ADDR_DF3

,USER_ID,ADDR1,ADDR2,ADDR_LEN
0,2654270,大庆路老运管处小区东楼东户,大庆路看循环,6
614,2669564,安徽省合肥市肥东县,南巷138号,6
618,2604037,旭日路永红一路97号301,旭日路文头岭,6
623,2178163,万科金品家园2幢1002,彩香花苑2撞,6
624,2768037,白松二路创业花园78栋402室,白松二路穿越,6
627,2276616,香樟路469号融科东南海ZH2栋3002室,香樟路468,6
632,2735479,康桥镇秀沿路3688号昌硕科技,康桥镇主要路,6
613,2669564,安徽省合肥市肥东县,南巷138号,6
633,2735479,康桥镇秀沿路3688号昌硕科技,康桥镇主要路,6
636,2538962,聚宝嘉苑南楼1单元301,聚宝公园31,6


In [234]:
import copy
dt = []
for user_id in set(list(ADDR_DF3.USER_ID)):
    addr_df = ADDR_DF3[ADDR_DF3.USER_ID==user_id]
    addr_df = addr_df.sort_values(by='ADDR_LEN',ascending=False)
    aa = list(addr_df.ADDR2)
    aaa = copy.deepcopy(aa)
    print(aaa)
    if addr_df.shape[0]>1:
        for i in range(len(aa)-1):
            for j in range(i+1,len(aa)):
                if difflib.SequenceMatcher(None,str(aa[i]),str(aa[j])).ratio()>0.3:
#                     print(aa[j])
                    if aa[j] in aaa:
                        aaa.remove(aa[j])
    
    dt.append((user_id,aaa))

['布龙路中百饰大楼一楼驴园', '布龙路中百饰大楼一楼驴', '布龙路中百饰大楼木', '布龙路中百饰大楼木', '布龙路中百饰']
['柯桥区大岗镇']
['新华路321号', '新华路321']
['大坪英利国际B栋3905', '大坪英利国际B董', '大坪英利国际2董']
['新洲北村122栋504室', '新洲北村122504', '新洲北村12250']
['浙江省哈关系厚实', '浙江省哈关系']
['祥和南区万隆公寓']
['东部星城西区']
['白沙镇中心医院', '白沙镇中心医']
['横岗街道华西新村六巷六号10', '横岗街道华西新村六巷']
['柯子岭12号']
['老一中往前300', '老一中往前30']
['长虹大道龙鑫']
['家和苑有开商']
['湖塘镇亚泰财富中心3029室', '湖塘镇亚泰财富中心3029', '湖塘镇亚泰财']
['清水路苑38栋2单元60', '清水路苑38栋']
['平洲上潮7:']
['黄石市黄石少一']
['东三环北路甲49', '东三环北路甲4']
['说差南竿败秸唔合６号', '说差南竿败秸唔合６']
['北海路999号', '北海路999']
['常平镇板石头']
['月浦庆安一村101号502']
['天府大道北段1']
['平西二支路28号201']
['大庆路看循环']
['新岸苑新岸路']
['玉山镇置地廣場']
['海泰居小区29号楼三单元401', '海泰居小区28号', '海泰居小区28']
['管庄东会西码头']
['香榭园林二期']
['北京大学第一医院', '北京大学第一医']
['十里河弘善家园211号1809门', '十里河弘善园家211号1809', '十里河弘善园家211号18', '十里河弘善家园211号0']
['元岭路晓翠园2栋40']
['浙江省湖州市无']
['官南大道长水机场', '官南大道长水']
['珠江路延安社区', '珠江路延安社']
['陶吴镇狮子山花苑7高楼601', '陶吴镇狮子山花苑7号楼601', '陶吴镇狮子山花苑7楼601', '陶吴镇狮子山花苑']
['福永镇大洋田']
['北岗新居小区']
['高陵马家湾米家岩']
['东兴街道东兴路']
['河南省林周师']
['玉河东街9号天和园小区', '玉河东街9号天和园小', '玉河东

In [235]:
ADDR_DF4 = pd.DataFrame(dt,columns=['USER_ID','ADDR3'])

In [236]:
ADDR_DF4

,USER_ID,ADDR3
0,2787328,[布龙路中百饰大楼一楼驴园]
1,2541569,[柯桥区大岗镇]
2,2473988,[新华路321号]
3,2568197,[大坪英利国际B栋3905]
4,2484228,[新洲北村122栋504室]
5,401418,[浙江省哈关系厚实]
6,2637836,[祥和南区万隆公寓]
7,2615315,[东部星城西区]
8,2482197,[白沙镇中心医院]
9,2150421,[横岗街道华西新村六巷六号10]


In [241]:
ADDR_DF5 = pd.merge(ADDR_DF3,ADDR_DF4,on='USER_ID',how='left').drop_duplicates(['USER_ID','ADDR1'])

In [244]:
addr_lists = []
for i in range(len(ADDR_DF5)):
    addr_list  = list(ADDR_DF5.ADDR3)[i]
    addr_list.append(list(ADDR_DF5.ADDR1)[i])
    addr_lists.append([len(addr_list),addr_list])

In [247]:
ADDR_DF5['addr_changed_cnt'] = [addr[0] for addr in addr_lists]
ADDR_DF5['addr_changed'] = [addr[1] for addr in addr_lists]

In [254]:
ADDR_DF6 = ADDR_DF5[['USER_ID','addr_changed_cnt','addr_changed']]

In [255]:
ADDR_DF7 = ADDR_DF1[ADDR_DF1.addr_cnt==1][['USER_ID','addr_cnt','ADDR']]
ADDR_DF7.columns=ADDR_DF6.columns

In [257]:
ADDR_DF8 = pd.concat([ADDR_DF7,ADDR_DF6],ignore_index=True)

In [260]:
ADDR_DF8.addr_changed_cnt.value_counts()

1    22089
2      746
3        6
Name: addr_changed_cnt, dtype: int64

In [261]:
#####################是否精确到详细地址####################################

In [266]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address'].sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')

In [269]:
bool_list = []
for addr in list(client_addr_df.ACTION_VALUE):
    c = 0
    for i in range(10):
        if str(i) in addr:
            c+=1
    if c>0:
        bool_list.append(1)
    else:
        bool_list.append(0)

In [271]:
client_addr_df['is_addr_accurate'] = bool_list

In [273]:
is_addr_accurate_df = client_addr_df[['USER_ID','is_addr_accurate']]

In [274]:
######################模糊详细地址##############################

In [275]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [278]:
client_addr_df1 = client_addr_df[['USER_ID','ACTION_VALUE']]

In [279]:
client_addr_df1['addr_len'] = client_addr_df1.ACTION_VALUE.str.len()

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [281]:
client_addr_df1[['ACTION_VALUE','addr_len']]

,ACTION_VALUE,addr_len
261174,亦庄经历开发区荣华中路汇丰公寓20,17.0
261175,亦庄经历开发区荣华中路汇丰公寓,15.0
261176,亦庄经历开发区荣华中路汇丰公寓2,16.0
261177,亦庄经历开发区荣华中路汇丰公寓1,16.0
261178,亦庄经历开发区荣华中路汇丰公寓11,17.0
261179,亦庄经历开发区荣华中路汇丰公寓116,18.0
261180,松陵城南花苑4,7.0
261181,松陵城南花苑41,8.0
261182,松陵城南花苑418,9.0
261183,松陵城南花苑418h,10.0


In [ ]:
#################是否使用默认输入法######################

In [282]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [344]:
ll = []
userlist = []
for USER_ID in set(list(client_addr_df.USER_ID)):
    ll.append(''.join([str(addr) for addr in list(client_addr_df[client_addr_df.USER_ID==USER_ID].ACTION_VALUE)]))
    userlist.append(USER_ID)

In [345]:
re.search(u"[a-z]+？+[a-z]",'zz？hz？hozhzhozhonzhongzhong？').group()

'zz？h'

In [346]:
is_default_input_list = []
for addrlist in ll:
    try :
        re.search(u"[a-z]+？+[a-z]",addrlist).group()
#         print(addrlist)
        is_default_input_list.append(1)
    except:
        is_default_input_list.append(0)

In [348]:
is_default_input_df = pd.DataFrame({'USER_ID':userlist,'is_default_input':is_default_input_list})

In [349]:
##############是否是手写输入#########################

In [350]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [351]:
client_addr_df.head()

,ACTION_CONTROL,ACTION_GROUP,ACTION_PAGE,ACTION_TEXT,ACTION_VALUE,CLIENT_TIME,CREATE_DATE,中文匹配,D3,D10,...,T30#,T3#,T10#,[T3->T10]%,T10%,T20#,TEL,USER_ID,日(APPLY_DATE),日(MIN_PUTOUTDATE)
261174,my_home_address,f29b4a00-6e24-44cd-a5cf-eaab0e322572,USER/contact,NaN,亦庄经历开发区荣华中路汇丰公寓20,2017/4/3 星期一 上午 9:59:37,2017/5/3 星期三 上午 10:00:03,NaN,1,1,...,0,0,0,NaN,0.00%,0,10391830691816441332,1742371,2017/2/10 星期五,2017/2/10 星期五
261175,my_home_address,f29b4a00-6e24-44cd-a5cf-eaab0e322572,USER/contact,NaN,亦庄经历开发区荣华中路汇丰公寓,2017/4/3 星期一 上午 9:59:38,2017/5/3 星期三 上午 10:00:03,NaN,1,1,...,0,0,0,NaN,0.00%,0,10391830691816441332,1742371,2017/2/10 星期五,2017/2/10 星期五
261176,my_home_address,f29b4a00-6e24-44cd-a5cf-eaab0e322572,USER/contact,NaN,亦庄经历开发区荣华中路汇丰公寓2,2017/4/3 星期一 上午 9:59:38,2017/5/3 星期三 上午 10:00:03,NaN,1,1,...,0,0,0,NaN,0.00%,0,10391830691816441332,1742371,2017/2/10 星期五,2017/2/10 星期五
261177,my_home_address,f29b4a00-6e24-44cd-a5cf-eaab0e322572,USER/contact,NaN,亦庄经历开发区荣华中路汇丰公寓1,2017/4/3 星期一 上午 9:59:40,2017/5/3 星期三 上午 10:00:03,NaN,1,1,...,0,0,0,NaN,0.00%,0,10391830691816441332,1742371,2017/2/10 星期五,2017/2/10 星期五
261178,my_home_address,f29b4a00-6e24-44cd-a5cf-eaab0e322572,USER/contact,NaN,亦庄经历开发区荣华中路汇丰公寓11,2017/4/3 星期一 上午 9:59:40,2017/5/3 星期三 上午 10:00:03,NaN,1,1,...,0,0,0,NaN,0.00%,0,10391830691816441332,1742371,2017/2/10 星期五,2017/2/10 星期五


In [ ]:
J,一,、,亠,乛

In [352]:
#############是否含有标点###################################

In [353]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [354]:
ll = []
userlist = []
for USER_ID in set(list(client_addr_df.USER_ID)):
    ll.append(''.join([str(addr) for addr in list(client_addr_df[client_addr_df.USER_ID==USER_ID].ACTION_VALUE)]))
    userlist.append(USER_ID)

In [355]:
is_obtain_someting_list = []
for l in ll:
    c = 0
    for a in ['(',')','（','）','【','】','[',']','{','}','-','_']:
        if a in l:
            c += 1
    if c == 0:
        is_obtain_someting_list.append(0)
    else:
        is_obtain_someting_list.append(1)
        
        

In [357]:
is_addr_include_punctuation = pd.DataFrame({'USER_ID':userlist,'is_default_input':is_obtain_someting_list})

In [358]:
###################地址长度最大值#########################################

In [359]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [360]:
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [363]:
max_addr_len_df = client_addr_df[['USER_ID','addr_str_len']].groupby('USER_ID').max().reset_index()

In [364]:
#################地址最终回落长度######

In [365]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [369]:
last_addr_len = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]

In [371]:
last_addr_len['last_addr_len'] = last_addr_len.ACTION_VALUE.str.len()

In [374]:
final_fall_addr_len = pd.merge(last_addr_len,max_addr_len_df,on='USER_ID',how='inner')

In [376]:
final_fall_addr_len['final_fall_addr_len'] = final_fall_addr_len.addr_str_len-final_fall_addr_len.last_addr_len

In [378]:
final_fall_addr_len_df = final_fall_addr_len[['USER_ID','final_fall_addr_len']]

In [380]:
####################地址最大回落长度################################

In [381]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [382]:
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [383]:
client_addr_df['addr_str_len_b'] = client_addr_df['addr_str_len'].shift(1)
client_addr_df['addr_str_len_a'] = client_addr_df['addr_str_len'].shift(-1)

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [395]:
def get_minmum(df):
    aa = []
    if len(df)>=3:
        for i in range(1,len(list(df.addr_str_len))-1):
            if (list(df.addr_str_len)[i]<list(df.addr_str_len)[i-1])&(list(df.addr_str_len)[i]<list(df.addr_str_len)[i+1]):
                aa.append(list(df.addr_str_len)[i])
            else:
                aa.append(9999)
    else:
        aa.append(9999)
    return min(aa)
max_fall_addr_len = client_addr_df.groupby('USER_ID').apply(get_minmum)

In [398]:
max_fall_addr_len = pd.merge(max_addr_len_df,max_fall_addr_len.reset_index(),on='USER_ID',how='inner')

In [399]:
max_fall_addr_len.head()

,USER_ID,addr_str_len,0
0,21807,10.0,5.0
1,26030,13.0,9.0
2,26047,7.0,3.0
3,26216,21.0,1.0
4,26298,23.0,9999.0


In [401]:
max_fall_addr_len['max_fall_addr_len'] = max_fall_addr_len.addr_str_len-max_fall_addr_len[0]

In [402]:
max_fall_addr_len.ix[max_fall_addr_len.max_fall_addr_len<0,'max_fall_addr_len'] = -1

In [405]:
max_fall_addr_len_df = max_fall_addr_len[['USER_ID','max_fall_addr_len']]

In [406]:
#############城区乡镇####################

In [407]:
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']

In [413]:
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]